# Predicting Home Runs

The goal is to predict the number of home runs MLB players will hit in the upcoming season, culminating in a league-wide leaderboard of predictions. The methodology involves:
- Data collection from FanGraphs 
- Preprocessing including feature engineering, standardization, and handling missing values
- Multiple linear regression and machine-learning model development
- Evaluation through Root Mean Squared Error (RMSE)
- Hyperparameter tuning and further improvements

## Import Libraries and Load Dataset
The dataset is sourced via the `pybaseball` library, pulling season-level batting stats and metrics from FanGraphs ranging from 2015 to 2024. Aside from the standard counting and rate statistics, advanced plate discipline and Statcast metrics like chase rate and exit velocity are also included. Using `pandas` to load the data into a DataFrame, the final dataset contains 6,370 observations across 320 columns with a total of 1,641 hitters. 

In [58]:
import pandas as pd
from pybaseball import batting_stats

# Batting statistics since 2015 for all positions
df = batting_stats(start_season=2015, end_season=2024, qual=1)

# Extract pitcher data and ids
df_pitchers = batting_stats(start_season=2015, end_season=2024, qual=1, position='p')
pitcher_ids = df_pitchers['IDfg'].unique()

# Filter out pitchers
df = df[~df['IDfg'].isin(pitcher_ids)]

columns = df.columns.to_list()
rows = df.shape[0]

print(f"Dataset Shape: {df.shape}")
print(f"Number of Players: {df['IDfg'].nunique()}")

Dataset Shape: (6370, 320)
Number of Players: 1641


## Dataset Preprocessing

In [59]:
df.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
145,15640,2024,Aaron Judge,NYY,32,158,559,704,180,85,...,117.5,238.0,0.609,391,0.146,0.267,NaN,NaN,NaN,11.3
157,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.4
321,25764,2024,Bobby Witt Jr.,KCR,24,161,636,709,211,123,...,116.9,259.0,0.481,538,0.138,0.236,NaN,NaN,NaN,10.0
166,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
172,10155,2018,Mike Trout,LAA,26,140,471,608,147,80,...,118.0,162.0,0.460,352,0.201,0.261,NaN,NaN,NaN,9.5


First, we will select a subset of 85 relevent, non-redundant features while renaming several columns to remove text like "(sc)" from the name. 

In [60]:
# Select relevant columns and omit repetitive columns
selected_columns = ['IDfg', 'Season', 'Name', 'Team', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B', 'HR', 'R', 'RBI',
                    'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG', 'GB', 'FB', 'LD', 'IFFB', 'Pitches',
                    'Balls', 'Strikes', 'IFH', 'BU', 'BUH', 'BB%', 'K%', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'BABIP',
                    'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'wOBA', 'wRAA', 'wRC', 'Bat', 'RAR',
                    'WAR', 'wRC+', 'O-Swing% (sc)', 'Z-Swing% (sc)', 'Swing% (sc)', 'O-Contact% (sc)', 'Z-Contact% (sc)',
                    'Contact% (sc)', 'Zone% (sc)', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%', 'EV', 'LA', 
                    'Barrels', 'Barrel%', 'maxEV', 'HardHit', 'HardHit%', 'Events', 'CStr%', 'CSW%', 'xBA', 'xSLG', 
                    'xwOBA', 'L-WAR']

# Select columns
df = df[selected_columns]

# Remove '(sc)' from plate discipline metrics
df.columns = df.columns.str.replace(r'\s*\(sc\)', '', regex=True)

print(f"Number of Selected Columns: {len(selected_columns)}")

Number of Selected Columns: 85


### Missing Values
Next, we will list columns with missing values and drop those with extensive missing data, ensuring a complete or mostly complete dataset for modeling. In this case, several observations appear to have missing values, including `xBA`, `xSLG`, and `xwOBA`, which have significant missing values. Since these metrics are calculated using batted-ball data like exit velocity (`EV`) and launch angle (`LA`), which are included in the dataset, we can remove these columns for now. 

In [61]:
missing_columns = df.isnull().sum()
missing_columns = missing_columns[missing_columns > 0]
missing_columns_list = missing_columns.to_dict()
print(missing_columns_list)

# Drop columns with completely missing data
df = df.drop(columns=['xBA', 'xSLG', 'xwOBA'])

# Drop missing observations
df = df.dropna()
new_rows = df.shape[0]

print(f"Observations Removed: {rows - new_rows}")

{'O-Swing%': 10, 'Z-Swing%': 3, 'Swing%': 2, 'O-Contact%': 44, 'Z-Contact%': 14, 'Contact%': 3, 'Zone%': 2, 'Pull%': 24, 'Cent%': 24, 'Oppo%': 24, 'Soft%': 24, 'Med%': 24, 'Hard%': 24, 'EV': 32, 'LA': 32, 'Barrels': 2, 'Barrel%': 26, 'maxEV': 32, 'HardHit': 2, 'HardHit%': 26, 'xBA': 6370, 'xSLG': 6370, 'xwOBA': 6370}
Observations Removed: 74


### Target Variable
Since we want to predict home runs for the following season, the target variable `HR_next` shifts each player’s home run count by one season, representing his home-run total for the next season.

In [62]:
# Sort by player and season to ensure consistency before shifting
df = df.sort_values(by=['IDfg', 'Season']).reset_index(drop=True)

# Create 'HR_next' by shifting HR column by -1 within each player group
df['HR_next'] = df.groupby('IDfg')['HR'].shift(-1).astype('Int64')

df[df['Name']=='Corey Seager'][['Season', 'HR', 'HR_next']].sort_values('Season')

,Season,HR,HR_next
3277,2015,4,26
3278,2016,26,22
3279,2017,22,2
3280,2018,2,19
3281,2019,19,15
3282,2020,15,16
3283,2021,16,33
3284,2022,33,33
3285,2023,33,30
3286,2024,30,<NA>


## Baseline Model
To establish a benchmark for future comparisons, an Ordinary Least Squares (OLS) regression model is built using the `statsmodels` library. This baseline model helps determine the predictive strength of our features without regularization or advanced methods.

### Training and Test Sets
We prepare the training and test datasets by:
- Dropping metadata and non-numeric columns
- Excluding rows with missing values in the target variable
- Splitting the training and test sets 80/20
- Standardizing each feature to ensure consistent scaling

### Standardization
Scaling the training set using `fit_transform()` prevents the model from "peeking" at the test data. Using `.transform()` on the test set applies the identical transformation, relying on the mean and standard deviation derived from the training set to maintain consistency between the two.

In [63]:
import numpy as np

# Drop irrelevant columns, keep only numeric columns, and drop rows with missing target variable
df_model = df.drop(columns=['IDfg', 'Name', 'Team']).select_dtypes(include=[np.number]).dropna(subset=['HR_next'])

# Separate the features and target
X = df_model.drop(columns=['HR_next'])
y = df_model['HR_next']

# Convert y to integer type if needed
y = y.astype(int)

# Split into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled arrays back to DataFrames with the original column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

# Reset the indices to align with y_train and y_test
X_train_scaled = X_train_scaled.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

### Model Fitting
The OLS model is fitted on the scaled training and target data, which produces the following summary output:

In [37]:
import statsmodels.api as sm

# Add a constant to X_train_scaled for statsmodels to include an intercept in the model
X_train_scaled = sm.add_constant(X_train_scaled)
X_test_scaled = sm.add_constant(X_test_scaled)

# Fit the model using statsmodels OLS and print the summary
ols_model = sm.OLS(y_train, X_train_scaled).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                HR_next   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.457
Method:                 Least Squares   F-statistic:                     42.34
Date:                Wed, 06 Nov 2024   Prob (F-statistic):               0.00
Time:                        19:53:59   Log-Likelihood:                -12742.
No. Observations:                3740   AIC:                         2.564e+04
Df Residuals:                    3663   BIC:                         2.612e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.7535      0.121     80.852      0.0

### Evaluation
The OLS regression results indicate that the model explains around 45.7% of the variance in next-season home runs (Adj. R-squared = 0.457), suggesting moderate predictive power. Significant predictors include Age (negative), wOBA, max exit velocity, and barrels. As expected, the model exhibits multicollinearity, meaning some predictors are highly correlated, which weakens model's reliability and accuracy. Furthermore, the Omnibus and Jarque-Bera diagnostic tests indicate some non-normality in residuals. Root mean squared error (RMSE) is used to gague the model's accuracy in predicting home runs, and the RMSE scores for the training and test sets are displayed below:

In [40]:
from sklearn.metrics import mean_squared_error

# RMSE Train
RMSE_train_ols = np.sqrt(mean_squared_error(y_train, ols_model.fittedvalues))

# RMSE Test
y_pred_ols = ols_model.predict(X_test_scaled)
RMSE_test_ols = np.sqrt(mean_squared_error(y_test, y_pred_ols))

print(f"RMSE Train: {RMSE_train_ols}")
print(f"RMSE Test: {RMSE_test_ols}")

RMSE Train: 7.301087628220538
RMSE Test: 7.334820855653133


The RMSE values for the train (7.30) and test (7.33) sets are quite close, indicating that the model generalizes reasonably well without significant overfitting. The RMSE of around 7.3 means that, on average, the model's predictions for next-season home run totals are off by about 7 home runs. These metrics establish a benchmark with which to compare subsequent models. Any model improvements should aim to reduce this error, enhancing predictive accuracy and demonstrating that the refinements add value over the simple OLS regression.

## Model Improvements

### Regularization
The regularization technique applied here is elastic net regression, which combines lasso (L1) and ridge (L2) regularization to enhance model performance by controlling for multicollinearity and reducing overfitting. Specifically:
- Elastic net parameters: The `alpha` parameter controls the overall strength of regularization, while `L1_wt` balances between L1 and L2 penalties
- Refitting: With `refit=True`, the model recalculates coefficients for the selected features post-regularization

In [41]:
X_train_scaled = X_train_scaled.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

reg_model = sm.OLS(y_train, X_train_scaled).fit_regularized(method='elastic_net', alpha=0.1, L1_wt=0.5, refit=True)

RMSE_train_reg = np.sqrt(mean_squared_error(y_train, reg_model.fittedvalues))

y_pred_reg = reg_model.predict(X_test_scaled)
RMSE_test_reg = np.sqrt(mean_squared_error(y_test, y_pred_reg))

print(f"Number of Features after Regularization: {sum(reg_model.params != 0)}")
print(f"RMSE Train: {RMSE_train_reg}")
print(f"RMSE Test: {RMSE_test_reg}")

Number of Features after Regularization: 30
RMSE Train: 7.484559414524934
RMSE Test: 7.503121710609439


Regularization did not reduce the training or test error, suggesting that regularization potentially removed relevant features. Adjusting the regularization strength or re-evaluating feature importance may help improve the results, however, the model's linear assumptions may not be capturing all the underlying relationships in the data. 

### Machine Learning

With the goal of accuracy in mind, we will use Extreme Gradient Boosting (XGBoost) – a powerful and efficient machine-learning algorithm well-suited for structured data.  

In [85]:
import xgboost as xgb

# Initialize the model
xgb_model = xgb.XGBRegressor(seed=42)

# Remove constant term from OLS
X_train_scaled = X_train_scaled.drop(columns='const', errors='ignore')
X_test_scaled = X_test_scaled.drop(columns='const', errors='ignore')

# Fit the model
xgb_model.fit(X_train_scaled, y_train)

y_pred_xgb_train = xgb_model.predict(X_train_scaled)
y_pred_xgb_test = xgb_model.predict(X_test_scaled)

RMSE_train_xgb = np.sqrt(mean_squared_error(y_train, y_pred_xgb_train))
RMSE_test_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb_test))

print(f"RMSE Train: {RMSE_train_xgb}")
print(f"RMSE Test: {RMSE_test_xgb}")

RMSE Train: 0.9362434894273352
RMSE Test: 7.485095468371326


The high discrepancy between training and test RMSE signals overfitting. To mitigate this, further hyperparameter tuning might be needed to improve generalization and achieve a more balanced performance on both datasets.

### Hyperparameter Tuning
We can perform hyperparameter tuning to further optimize the model using `GridSearchCV` to fine-tune several parameters:
- `max_depth`: Controls tree depth and increases complexity
- `n_estimators`: Number of trees to fit
- `learning_rate`: Controls how quickly the model adapts
- `min_split_loss`: Minimum loss reduction to split a node
- `subsample`: Fraction of samples used for training each tree, introducing randomness

Multiple XGBoost models are trained with various combinations of these parameters, and each model is evaluated using 3-fold cross-validation, resulting in a trained model with the "best" set of parameters. 

In [87]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for tuning
param_grid = {
    'max_depth': [3, 6],
    'n_estimators': [100, 1000],
    'learning_rate': [0.01, 0.1, 0.3],
    'min_split_loss': [0, 1, 5],
    'subsample': [0.5, 1],
}

# Initialize new XGBRegressor with no prior parameters
xgb_model = xgb.XGBRegressor(seed=42)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Fit GridSearchCV
grid_search.fit(X_train_scaled, y_train)

# Best parameters and model
best_params = grid_search.best_params_
xgb_model_tuned = grid_search.best_estimator_

print(f"Best parameters found: {best_params}")
print(f"Cross-Validated RMSE: {-grid_search.best_score_}")

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters found: {'learning_rate': 0.01, 'max_depth': 3, 'min_split_loss': 5, 'n_estimators': 1000, 'subsample': 0.5}
Cross-Validated RMSE: 7.237471947066115


The cross-validation process tested 72 combinations of hyperparameters for the XGBoost model, each evaluated across three folds, resulting in 216 total fits. The optimal parameters found were:

- `learning_rate`: 0.01, slowing down learning for more gradual improvements
- `max_depth`: 3, limiting the depth of each tree and helping to avoid overfitting
- `min_split_loss`: 5, requiring a minimum loss reduction for a node to split
- `n_estimators`: 1000, a large number of boosting rounds to ensure model convergence
- `subsample`: 0.5, using only half of the training samples for each tree improved generalization.

The cross-validated RMSE of 7.24 shows that tuning has reduced overfitting and slightly increased the model’s predictive accuracy on unseen data. Below are the RMSE training and test scores for each model:

In [88]:
# Predictions using the best model
y_pred_train_tuned = xgb_model_tuned.predict(X_train_scaled)
y_pred_test_tuned = xgb_model_tuned.predict(X_test_scaled)

# Calculate RMSE for the best model
RMSE_train_xgb_tuned = np.sqrt(mean_squared_error(y_train, y_pred_train_tuned))
rmse_test_xgb_tuned = np.sqrt(mean_squared_error(y_test, y_pred_test_tuned))

print(f"Baseline RMSE Train: {RMSE_train_ols}")
print(f"Baseline RMSE Test: {RMSE_test_ols}\n")
print(f"Tuned XGBoost RMSE Train: {RMSE_train_xgb_tuned}")
print(f"Tuned XGBoost RMSE Test: {rmse_test_xgb_tuned}\n")
print(f"Untuned XGBoost RMSE Train: {RMSE_train_xgb}")
print(f"Untuned XGBoost RMSE Test: {RMSE_test_xgb}\n")
print(f"Regularized OLS RMSE Train: {RMSE_train_reg}")
print(f"Regularized OLS RMSE Test: {RMSE_test_reg}")

Baseline RMSE Train: 7.301087628220538
Baseline RMSE Test: 7.334820855653133

Tuned XGBoost RMSE Train: 6.071578793005435
Tuned XGBoost RMSE Test: 7.05256944659387

Untuned XGBoost RMSE Train: 0.9362434894273352
Untuned XGBoost RMSE Test: 7.485095468371326

Regularized OLS RMSE Train: 7.484559414524934
Regularized OLS RMSE Test: 7.503121710609439


It's evident that the tuned XGBoost model demonstrates the best balance between training and test errors, reducing overall prediction error and outperforming both the baseline and other alternatives in terms of generalization. This model appears to capture relevant patterns most effectively without overfitting, making it the strongest candidate to generate the projected home run leaderboard.
## Projected Leaderboard

In [95]:
df_2024 = df[df['Season']==2024].copy()

features = X.columns.tolist()

X_2024 = df_2024[features]
X_2024_scaled = scaler.transform(X_2024)

df_2024['Proj HR'] = xgb_model_tuned.predict(X_2024_scaled).astype(int)

leaderboard = df_2024[['Name', 'Proj HR']].sort_values(by='Proj HR', ascending=False).reset_index(drop=True)

leaderboard.head(10)

,Name,Proj HR
0,Shohei Ohtani,38
1,Juan Soto,33
2,Aaron Judge,33
3,Bobby Witt Jr.,32
4,Anthony Santander,31
5,Yordan Alvarez,31
6,Vladimir Guerrero Jr.,29
7,Jose Ramirez,29
8,Giancarlo Stanton,28
9,Ketel Marte,28


## Further Improvements
While the tuned XGBoost model shows promising performance, there are additional steps that could further enhance its accuracy and robustness:
- **Feature Engineering**: Adding or refining features could reveal new patterns relevant to home runs. For instance, creating interaction terms or transforming certain features (e.g. Age^2) might improve the model’s ability to capture underlying, nonlinear trends. Other biological factors, like height and weight, or experience level with relation to age could also be included. Bat-tracking data could also prove relevant to power production.

- **Time-Based Validation**: Since player performance can vary over time, implementing a time-series cross-validation approach would ensure that the model captures temporal dependencies between seasons better.

- **Alternate Ensemble/Deep Learning Methods**: Other algorithms, such as random forests or neural networks, may enhance predictive performance by leveraging each model’s strengths.

- **Regularization Tuning**: Fine-tuning regularization parameters within XGBoost, such as L1 and L2 penalties, could help reduce overfitting further. Adjusting these parameters would add more control over the complexity of the model, promoting better generalization.

- **Handling Seasonal and Tracking System Changes**: Since MLB tracking systems have evolved (e.g., from Trackman to Hawkeye), including adjustments for these transitions or isolating data by tracking system might help reduce noise introduced by measurement changes.

- **Minor League Statistics**: Rookies who have yet to play in the prior season could be incorporated into the model using their minor league statistics, adjusted for the level of play. 